In [3]:
import os
import asyncio
from typing import Optional

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableParallel, RunnablePassthrough
from dotenv import load_dotenv

# --- Configuration ---
# Ensure your API key environment variable is set (e.g., OPENAI_API_KEY)
load_dotenv()
try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(model="gemini-2.5-flash", temperature=0.7, base_url=os.getenv('OPENAI_API_BASE'), api_key=os.getenv('OPENAI_API_KEY'))
    if llm:
        print(f"Language model initialized: {llm.model_name}")
except Exception as e:
    print(f"Error initializing language model: {e}")
    llm = None


# --- Define Independent Chains ---
# These three chains represent distinct tasks that can be executed in parallel.

summarize_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Summarize the following topic concisely:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

questions_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Generate three interesting questions about the following topic:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)

terms_chain: Runnable = (
    ChatPromptTemplate.from_messages([
        ("system", "Identify 5-10 key terms from the following topic, separated by commas:"),
        ("user", "{topic}")
    ])
    | llm
    | StrOutputParser()
)


# --- Build the Parallel + Synthesis Chain ---

# 1. Define the block of tasks to run in parallel. The results of these,
#    along with the original topic, will be fed into the next step.
map_chain = RunnableParallel(
    {
        "summary": summarize_chain,
        "questions": questions_chain,
        "key_terms": terms_chain,
        "topic": RunnablePassthrough(),  # Pass the original topic through
    }
)

# 2. Define the final synthesis prompt which will combine the parallel results.
synthesis_prompt = ChatPromptTemplate.from_messages([
    ("system", """Based on the following information:
     Summary: {summary}
     Related Questions: {questions}
     Key Terms: {key_terms}
     Synthesize a comprehensive answer."""),
    ("user", "Original topic: {topic}")
])

# 3. Construct the full chain by piping the parallel results directly
#    into the synthesis prompt, followed by the LLM and output parser.
full_parallel_chain = map_chain | synthesis_prompt | llm | StrOutputParser()


# --- Run the Chain ---
async def run_parallel_example(topic: str) -> None:
    """
    Asynchronously invokes the parallel processing chain with a specific topic
    and prints the synthesized result.

    Args:
        topic: The input topic to be processed by the LangChain chains.
    """
    if not llm:
        print("LLM not initialized. Cannot run example.")
        return

    print(f"\n--- Running Parallel LangChain Example for Topic: '{topic}' ---")
    try:
        # The input to `ainvoke` is the single 'topic' string, which is
        # then passed to each runnable in the `map_chain`.
        response = await full_parallel_chain.ainvoke(topic)
        print("\n--- Final Response ---")
        print(response)
    except Exception as e:
        print(f"\nAn error occurred during chain execution: {e}")

if __name__ == "__main__":
    test_topic = "北京的历史和文化"
    # In Jupyter notebooks, an event loop is already running.
    # We should use await directly instead of asyncio.run()
    await run_parallel_example(test_topic)

Language model initialized: gemini-2.5-flash

--- Running Parallel LangChain Example for Topic: '北京的历史和文化' ---

--- Final Response ---
北京，作为一座拥有三千多年建城史和八百多年建都史的古老都市，其历史与文化深度融合，形成了独一无二的城市魅力。它不仅是元、明、清三朝及中华人民共和国的首都，更是中华文明的集大成者，以其宏大庄重、兼容并蓄、古老与现代并存的特征，深刻塑造了中国的物质与非物质文化。

### 一、 悠久的历史沿革与深远影响

北京的历史地位，作为中国多个朝代的都城，深刻塑造了这座城市的方方面面。从古代的蓟城、燕国都城，到辽金的陪都，再到元大都、明清北京城，直至新中国的首都，北京始终是中国北方乃至全国的政治、军事与文化中心。

这种长期作为政治中心的地位，首先体现在其**物质文化**上：
*   **宏伟的皇家建筑群**：故宫、天坛、长城、颐和园等**皇家园林**，无不彰显着皇权的至高无上和国家的威严。这些建筑不仅是精湛工艺的结晶，更是礼制、宇宙观和美学思想的物质载体。它们的宏大庄重，直接反映了作为都城的政治需求和文化品味。
*   **独特的民居形式**：与皇家建筑相辅相成的是**胡同**和**四合院**。它们构成了北京独特的城市肌理，体现了古代城市规划的智慧和百姓的居住哲学，承载着邻里情谊和传统生活方式。
*   **丰富的艺术形式**：为了满足宫廷和上层社会的需求，京剧、景泰蓝、雕漆、宫廷画等艺术门类在北京发展至巅峰，成为中华传统艺术的代表。这些艺术品不仅是美的享受，更是等级制度、审美情趣和文化交流的体现。

同时，北京的都城地位也深深塑造了其**非物质文化**：
*   **礼仪与价值观**：作为礼仪之邦的都城，北京的社会生活深受儒家思想和宫廷礼仪的影响，形成了注重规矩、讲究体面、尊老爱幼的社会风气。其“宏大庄重”的城市气质也内化为市民的行为规范和价值观。
*   **语言特色**：北京话作为中国普通话的基础，其发音、词汇和表达方式，因其作为京城语言的特殊地位，被广泛传播和接受，成为国家标准语的基石。
*   **思维方式**：作为最高学府聚集地和思想文化交流的中心，北京一直是各种思潮碰撞、融合之地，塑造了开放、包容、